# Local execution
Sometimes (often) you want to check how your pipeline is working locally before submitting it.

Here's the manual way of creating the pipeline and executing it.

At first, install the netunicorn-executor.

In [1]:
%pip install netunicorn-executor

Note: you may need to restart the kernel to use updated packages.


If you do it in the already running asyncio loop (e.g., in Jupyter), you also need to install and import nest_asyncio. This is not required if you run it as a Python script.

In [2]:
%pip install nest-asyncio

Note: you may need to restart the kernel to use updated packages.


In [3]:
import nest_asyncio
nest_asyncio.apply()

Create your pipeline. We'll use a simple one for that example.

In [4]:
from netunicorn.base import Pipeline, Task

class DummyTask(Task):
    def run(self):
        return 0

class DummyRepeaterTask(Task):
    def __init__(self, lookup_for: str):
        self.lookup_for = lookup_for
        super().__init__()

    def run(self):
        previous_results = self.previous_steps[self.lookup_for]
        return f"I've heard of {previous_results}!"

pipeline = Pipeline().then(DummyTask(name="dummy1")).then(DummyRepeaterTask(lookup_for='dummy1'))

Then, import and create an executor with fake parameters.

Also, set `pipeline.report_results` to False (to prevent reporting results to our fake gateway), assign pipeline to executor.execution_graph, and manually set the executor state to EXECUTING

In [5]:
from netunicorn.executor import Executor, ExecutorState

executor = Executor(executor_id="local", gateway_endpoint="fake", experiment_id="local", heartbeat=False)
pipeline.report_results = False
executor.execution_graph = pipeline
executor.state = ExecutorState.EXECUTING

INFO:executor_local:Parsed configuration: Gateway located on fake
INFO:executor_local:Current directory: /home/kell/repos/netunicorn/examples


Or, instead, you can call the `get_local_executor` function from the `netunicorn.executor` module. It will do all the above steps for you and return the executor.

In [6]:
from netunicorn.executor import get_local_executor
executor = get_local_executor(pipeline)  # it implements all the above steps

INFO:executor_local:Parsed configuration: Gateway located on fake
INFO:executor_local:Current directory: /home/kell/repos/netunicorn/examples


And then, call the executor to start running the pipeline.

In [7]:
executor()

INFO:executor_local:Execution is finished, start reporting results.
INFO:executor_local:Skipping reporting results due to execution graph setting.


Now you can explore locally create log file and `executor.pipeline_results`.

In [8]:
print(executor.execution_graph_results)

<Success: defaultdict(<class 'list'>, {'dummy1': [<Success: 0>], 'a71d8ec1-62bf-468b-a234-9af9e5f18ecd': [<Success: I've heard of [<Success: 0>]!>]})>
